## First Attempt of Shear Profile of A360 with metadetection

Contact author: Miranda Gorsuch

First attempt at creating a shear profile for A360 using cell-based coadds and `metadetection`. Many parts are from the [Shear profile around A360 using ComCam HSM shapes](https://github.com/lsst-sitcom/comcam_clusters/blob/main/ACO360_WL_HSCcalib_CLMM.ipynb) notebook, especially the identification of red sequence galaxies and use of CLMM to create the tangential shear plot.

Last working weekly: `w_2025_24`

Container Size: small (4 GB)

This is not a finished notebook and requires a lot of work yet. This is to show that the end-to-end for the generation of cell-based coadds, running `metadetection`, calibrating shears, and calculating tangential shears is almost in place.

Things to check / improve / fix
- [X] ~~Generate cell-based coadds in *g* band~~
- [X] ~~Better identification and removal of RS galaxies with *g*-band~~
- [ ] Incorporate basic photo-z information
- [X] ~~Removal of duplicate objects from tract / patch overlap~~
- [ ] Compute shape errors from `metadetect`
- [ ] Null tests
- [X] ~~Check where factor of 2 is needed for e to g conversion (shapes should already be in g)~~

# Preparing data

## Imports & Definitions

In [ ]:
# locally install modeling packages (only do once, if not already installed)
# pip install pyccl
# pip install clmm

In [ ]:
from lsst.daf.butler import Butler

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib

import astropy.units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord

import gc

from lsst.skymap import Index2D
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath
import lsst.geom as geom

from numpy.linalg import inv
import scipy.stats as stats

import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData, Cosmology
from clmm import Cosmology, utils

cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

%matplotlib inline

In [ ]:
REPO = '/sdf/data/rubin/repo/main/'
butler = Butler(REPO)
registry = butler.registry

# collection = 'u/mgorsuch/metadetect/a360/20250513T210342Z' # metadetect run on two bands
collection = 'u/mgorsuch/metadetect/a360_3_band/20250612T151357Z' # metadetect run on g, r, i
cell_collection = 'u/mgorsuch/a360_cell_coadd/20250513T044026Z' # cells used as input for r and i
cell_collection_g = 'u/mgorsuch/a360_cell_coadd_g/20250611T144112Z' # cells used as input for g

## Read in data

Metadetect outputs tables for each patch. Read in each table and compile them together.

In [ ]:
# Position of the BCG for A360
ra_bcg = 37.862
dec_bcg = 6.98

The cell below is for finding the tracts/patches that are within the specified radius of the BCG. This is already incorporated in the butler collection used for the `metadetect` output.

In [ ]:
# skymap = butler.get('skyMap', skymap='lsst_cells_v1', collections='LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810')

# # Looking for all patches in delta deg region around it
# delta = 0.5
# center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
# ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
# dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

# ra_range = (ra_min, ra_max)
# dec_range = (dec_min, dec_max)
# radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
#          geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
#          geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
#          geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

# tractPatchList = skymap.findTractPatchList(radec)

# find dataset refs that are within the tract/patch list above
# datasetRefs_shear = []
# datasetRefs_catalog = []

# for tractPatch in tractPatchList:
#     tract = tractPatch[0]
#     patchInfo = tractPatch[1]
#     for patch in patchInfo:
#         datasetRefs_shear.append(butler.query_datasets('ShearObject', 
#                                                  collections=collection,
#                                                  tract=tract.tract_id,
#                                                  patch=patch.sequential_index))
#         datasetRefs_catalog.append(butler.query_datasets('objectTable', 
#                                                  collections=default_collection,
#                                                  tract=tract.tract_id,
#                                                  patch=patch.sequential_index))

In [ ]:
datasetRefs_shear = []
overlap_patches_10463 = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
tract_patch_list = [] # only used for plotting distribution of input images

for ref in butler.registry.queryDatasets('ShearObject', collections=collection):

    if ref.dataId['tract'] == 10704 or ref.dataId['tract'] == 10705:
        continue

    if ref.dataId['tract'] == 10463 and ref.dataId['patch'] in overlap_patches_10463:
        continue
    
    datasetRefs_shear.append(butler.query_datasets('ShearObject', 
                                                     collections=collection,
                                                     skymap = 'lsst_cells_v1',
                                                     tract=ref.dataId['tract'],
                                                     patch=ref.dataId['patch']))

    tract_patch_list.append([ref.dataId['tract'], ref.dataId['patch']])

In [ ]:
shear_table_list = []

for i, ref in enumerate(datasetRefs_shear):
    shear_data_patch = butler.get(ref[0])
    shear_table_patch = shear_data_patch.to_pandas()
    shear_table_list.append(shear_table_patch)

shear_table = pd.concat(shear_table_list)

# remove unused tables to clear up memory
del shear_table_list
gc.collect()

In [ ]:
# remove objects in outer ring of cells in each patch since patch overlap is two cells
shear_table = shear_table[shear_table['cell_x']!=0]
shear_table = shear_table[shear_table['cell_x']!=21]
shear_table = shear_table[shear_table['cell_y']!=0]
shear_table = shear_table[shear_table['cell_y']!=21]
print("Number of rows after removing most duplicate cells: ", len(shear_table))

filt1 = shear_table['tract'] == 10463
filt1 &= shear_table['patch_x'] == 0
filt1 &= shear_table['cell_x'] == 1
shear_table = shear_table[np.invert(filt1)]

filt2 = shear_table['tract'] == 10463
filt2 &= shear_table['patch_x'] == 0
filt2 &= shear_table['cell_x'] == 2
shear_table = shear_table[np.invert(filt2)]
print("Number of rows after removing patch overlap in 10463: ", len(shear_table))

# some patch overlap appears to have a 6(?) cell overlap
filt3 = shear_table['tract'] == 10464
filt3 &= shear_table['patch_x'] == 9
filt3 &= shear_table['cell_x'] == 20
shear_table = shear_table[np.invert(filt3)]

# some patch overlap appears to have a 4 cell overlap
filt4 = shear_table['tract'] == 10464
filt4 &= shear_table['patch_x'] == 9
filt4 &= shear_table['cell_x'] == 19
shear_table = shear_table[np.invert(filt4)]
print("Number of rows after removing patch overlap in 10464: ", len(shear_table))

In [ ]:
# remove exact duplicates   
print("Number of rows prior to removing duplicates: ", len(shear_table))
shear_table= shear_table.drop_duplicates(subset=['shear_type', 'ra', 'dec']) # each object will potentially have several sheared images
print("Number of rows after removing duplicates: ", len(shear_table))

### Add useful columns

In [ ]:
# make new columns to convert nJy fluxes to AB magnitudes
t1 = Table.from_pandas(shear_table)

t1['wmom_band_mag_g'] = (t1['wmom_band_flux_g']*u.nJy).to(u.ABmag)
t1['wmom_band_mag_r'] = (t1['wmom_band_flux_r']*u.nJy).to(u.ABmag)
t1['wmom_band_mag_i'] = (t1['wmom_band_flux_i']*u.nJy).to(u.ABmag)
t1['wmom_color_mag_g-r'] = t1['wmom_band_mag_g']-t1['wmom_band_mag_r']
t1['wmom_color_mag_g-i'] = t1['wmom_band_mag_g']-t1['wmom_band_mag_i']
t1['wmom_color_mag_r-i'] = t1['wmom_band_mag_r']-t1['wmom_band_mag_i']

shear_table = t1.to_pandas()

In [ ]:
# Add columns for distance from BCG
c1 = SkyCoord(shear_table['ra'].values*u.deg, shear_table['dec'].values*u.deg)
c2 = SkyCoord(ra_bcg*u.deg, dec_bcg*u.deg)
sep = c1.separation(c2)

shear_table['deg_sep'] = sep.value

shear_table['mpc_sep'] = cosmo.eval_da(0.22) * shear_table['deg_sep'] * np.pi/180

In [ ]:
# polar angle of source galaxy relative to BCG, from -pi to pi
shear_table['phi'] = np.arctan2(shear_table['dec'] - dec_bcg, (ra_bcg - shear_table['ra'])*np.cos(np.deg2rad(dec_bcg)))

In [ ]:
# calculate tangential / cross shear components for each galaxy
# note that these shears need to be selected for ONLY the 'ns' (no artificial shear type)
shear_table['shear_t'] = -shear_table['wmom_g_1'] * np.cos(2*shear_table['phi']) \
                                - shear_table['wmom_g_2'] * np.sin(2*shear_table['phi'])
shear_table['shear_x'] = shear_table['wmom_g_1'] * np.sin(2*shear_table['phi']) \
                                - shear_table['wmom_g_2'] * np.cos(2*shear_table['phi'])

### Plot distribution of input images per cell in A360 region

Each run of the `get_cell_inputs` function should only take 2-3 minutes. If it's longer, try again later.

Note that there is a warning for each coadd patch read in: `lsst.cell_coadds._fits WARNING: Unable to read aperture correction map from the file.` This is OK to ignore, it's just stating that these coadds don't have aperture correction information.

In [ ]:
from lsst.sphgeom import Box, HealpixPixelization
import healsparse as hsp
import healpy as hp
from hpgeom import hpgeom
import skyproj

In [ ]:
# define healpix parameters
nside_coverage = 2**8
nside_sparse = 2**14

pixelization = HealpixPixelization(hp.nside2order(nside_sparse))

In [ ]:
# method find the pixel indices that overlap the sky projection of the cell area
def get_cell_pixels(cell, wcs):
    cell_bbox = cell.inner.bbox
    begin_coord = wcs.pixelToSky(cell_bbox.beginX, cell_bbox.beginY)
    end_coord = wcs.pixelToSky(cell_bbox.endX, cell_bbox.endY)
    
    if begin_coord.getRa() < end_coord.getRa():
        ra1 = begin_coord.getRa().asDegrees()
        ra2 = end_coord.getRa().asDegrees()
    else:
        ra1 = end_coord.getRa().asDegrees()
        ra2 = begin_coord.getRa().asDegrees()
    
    if begin_coord.getDec() < end_coord.getDec():
        dec1 = begin_coord.getDec().asDegrees()
        dec2 = end_coord.getDec().asDegrees()
    else:
        dec1 = end_coord.getDec().asDegrees()
        dec2 = begin_coord.getDec().asDegrees()

    indices = hpgeom.query_box(nside=nside_sparse, a0=ra1, a1=ra2, b0=dec1, b1=dec2)
    
    return indices

In [ ]:
def get_cell_inputs(cell_collection, tract_patch_list, band):

    cell_df = pd.DataFrame()
    cell_ra = []
    cell_dec = []
    pixel_indices = []
    inputs_list = []

    segs = [] # collection of lines to plot patch outlines

    for tract, patch in tract_patch_list:
    
        coadd = butler.get('deepCoaddCell', 
                         collections = cell_collection, 
                         instrument = 'LSSTComCam', 
                         skymap = 'lsst_cells_v1', 
                         tract = tract, 
                         patch = patch,
                         band = band,)
        # define a wcs from the given coadd
        wcs = coadd.wcs

        # get coadd outline
        coadd_corners = coadd.inner_bbox.getCorners()
    
        for index, corner in enumerate(coadd_corners):
            corner_coord_start = wcs.pixelToSky(corner.getX(), corner.getY())
            if index < 3:
                corner_coord_end = wcs.pixelToSky(coadd_corners[index+1].getX(), coadd_corners[index+1].getY())
            else:
                corner_coord_end = wcs.pixelToSky(coadd_corners[0].getX(), coadd_corners[0].getY())
    
            start_ra = corner_coord_start[0].asDegrees()
            start_dec = corner_coord_start[1].asDegrees()
    
            end_ra = corner_coord_end[0].asDegrees()
            end_dec = corner_coord_end[1].asDegrees()
    
            segs.append(((start_ra, start_dec), (end_ra, end_dec)))
        
        cell_list = list(coadd.cells.keys()) # skips indices that are empty

        # for each cell in cell_list:
        for index, cell_index in enumerate(cell_list):
    
            cell = coadd.cells[cell_index]
    
            # get cell coordinates for removing duplicates
            cell_center = cell.inner.bbox.getCenter()
            cell_center_coord = wcs.pixelToSky(cell_center)
            cell_ra.append(cell_center_coord.getRa().asDegrees())
            cell_dec.append(cell_center_coord.getDec().asDegrees())
        
            pixel_indices.append(get_cell_pixels(cell, wcs))
    
            inputs_list.append(cell.visit_count)
    
        del coadd
        gc.collect()
    
    cell_df["ra"] = cell_ra
    cell_df["dec"] = cell_dec
    cell_df["pixels"] = pixel_indices
    cell_df["inputs"] = inputs_list

    return cell_df, segs

#### g-band

In [ ]:
cell_df_g, segs_g = get_cell_inputs(cell_collection_g, tract_patch_list, 'g')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_g = cell_df_g.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_g = cell_df_g.explode('pixels').reset_index(drop=True)
pixel_df_g = pixel_df_g.drop_duplicates(subset=["pixels"]) 
pixel_df_g = pixel_df_g.dropna(subset=['pixels'])

pixels_g = pixel_df_g["pixels"].to_numpy()
pixel_input_g = pixel_df_g["inputs"].to_numpy()

In [ ]:
hsp_map_input_g = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_g.update_values_pix(np.array(pixels_g, dtype=np.int64), np.array(pixel_input_g))

#### i-band

In [ ]:
cell_df_i, segs_i = get_cell_inputs(cell_collection, tract_patch_list, 'i')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_i = cell_df_i.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_i = cell_df_i.explode('pixels').reset_index(drop=True)
pixel_df_i = pixel_df_i.drop_duplicates(subset=["pixels"]) 
pixel_df_i = pixel_df_i.dropna(subset=['pixels'])

pixels_i = pixel_df_i["pixels"].to_numpy()
pixel_input_i = pixel_df_i["inputs"].to_numpy()

In [ ]:
hsp_map_input_i = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_i.update_values_pix(np.array(pixels_i, dtype=np.int64), np.array(pixel_input_i))

#### r-band

In [ ]:
cell_df_r, segs_r = get_cell_inputs(cell_collection, tract_patch_list, 'r')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_r = cell_df_r.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_r = cell_df_r.explode('pixels').reset_index(drop=True)
pixel_df_r = pixel_df_r.drop_duplicates(subset=["pixels"]) 
pixel_df_r = pixel_df_r.dropna(subset=['pixels'])

pixels_r = pixel_df_r["pixels"].to_numpy()
pixel_input_r = pixel_df_r["inputs"].to_numpy()

In [ ]:
hsp_map_input_r = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_r.update_values_pix(np.array(pixels_r, dtype=np.int64), np.array(pixel_input_r))

#### Input Distribution Plot

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 5.3))
sp_g = skyproj.GnomonicSkyproj(ax=ax[0], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_g.draw_hspmap(hsp_map_input_g)
sp_g.ax.set_xlabel("RA", fontsize=10,)
sp_g.ax.set_ylabel("DEC", fontsize=10,)
sp_g.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_g.ax.set_title("g-band", pad=25)
for seg in segs_g:
    sp_g.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)
    
sp_i = skyproj.GnomonicSkyproj(ax=ax[1], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_i.draw_hspmap(hsp_map_input_i)
sp_i.ax.set_xlabel("RA", fontsize=10,)
sp_i.ax.set_ylabel("DEC", fontsize=10,)
sp_i.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_i.ax.set_title("i-band", pad=25)
for seg in segs_i:
    sp_i.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

sp_r = skyproj.GnomonicSkyproj(ax=ax[2], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_r.draw_hspmap(hsp_map_input_r)
sp_r.ax.set_xlabel("RA", fontsize=10,)
sp_r.ax.set_ylabel("DEC", fontsize=10,)
sp_r.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_r.ax.set_title("r-band", pad=25)
for seg in segs_r:
    sp_r.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

plt.suptitle("Cell Input Distribution of the A360 Region")
plt.show()

The three missing patches were due to pipeline failures, though are not included within the 0.5 degree radius of the BCG

## Apply `metadetect` flags

Anything that is flagged should be removed.

In [ ]:
print(shear_table['wmom_flags'].unique())
print(shear_table['psfrec_flags'].unique())
print(shear_table['wmom_psf_flags'].unique())
print(shear_table['wmom_obj_flags'].unique())
print(shear_table['wmom_T_flags'].unique())
print(shear_table['wmom_band_flux_flags_r'].unique())
print(shear_table['wmom_band_flux_flags_i'].unique())

In [ ]:
meta_filter = shear_table['wmom_flags']==False
meta_filter &= shear_table['psfrec_flags']==False
meta_filter &= shear_table['wmom_psf_flags']==False
meta_filter &= shear_table['wmom_obj_flags']==False
meta_filter &= shear_table['wmom_T_flags']==False
meta_filter &= shear_table['wmom_band_flux_flags_r']==False
meta_filter &= shear_table['wmom_band_flux_flags_i']==False

shear_table = shear_table[meta_filter]

print("Number of rows after removing metadetect flags: ", len(shear_table))

## Identify and remove cluster member galaxies

In [ ]:
# isolate no shear catalog
shear_table_ns = shear_table[shear_table['shear_type']=='ns']

### Color-magnitude diagrams

Identify red sequence by eye for now.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(8,4))
ax[0].hist(shear_table_ns['wmom_band_mag_g'], bins=75)
ax[1].hist(shear_table_ns['wmom_band_mag_r'], bins=75)
ax[2].hist(shear_table_ns['wmom_band_mag_i'], bins=75)
ax[0].set_yscale('log')
ax[1].set_yscale('log')
ax[2].set_yscale('log')

### Visual Inspections

Cuts are applied to all shear type catalogs, though only non-sheared are shown

In [ ]:
# Filter objects to within < 0.1 deg of cluster center
filt = shear_table['deg_sep'] < 0.1 # stay close to cluster center for RS indentification
shear_table_rs = shear_table[shear_table['deg_sep'] < 0.1] # catalog for RS identification
print("Number of rows in after applying < 0.1 deg from center for all shear types: ", len(shear_table_rs))

In [ ]:
# plotting parameters
point_size = 0.8
point_alpha = 0.7

r_left = 19
r_right = 23
r_down_left = 0.4
r_up_left = 0.6
r_down_right = 0.3
r_up_right = 0.5
r_slope = (r_down_right - r_down_left) / (r_right - r_left)

g_left = 21.5
g_right = 23
g_down_left = 1.7
g_up_left = 2
g_down_right = 1.4
g_up_right = 1.7
g_slope = (g_down_right - g_down_left) / (g_right - g_left)

In [ ]:
shear_table_rs_ns = shear_table_rs[shear_table_rs['shear_type']=='ns']

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_rs_ns['wmom_band_mag_r'], shear_table_rs_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size, alpha=point_alpha)
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_rs_ns['wmom_band_mag_g'], shear_table_rs_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size, alpha=point_alpha)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([20,25])

plt.show()

Note that combining all catalogs causes a "smeary" look is due to the 5 sheared/unsheared images of the same object that are detected & measured in slightly different ways.

### Filter RS galaxies

In [ ]:
rs_hi_ri = r_up_left + r_slope * (shear_table_rs['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_rs['wmom_band_mag_r']-r_left)
ri_filt = shear_table_rs['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_rs['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_rs['wmom_band_mag_r'] < 23

rs_hi_gi = g_up_left + g_slope * (shear_table_rs['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_rs['wmom_band_mag_g']-g_left)
gi_filt = shear_table_rs['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_rs['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_rs['wmom_band_mag_g'] < 23

# ns only, for plotting
ri_filt_ns = np.logical_and(ri_filt, shear_table_rs['shear_type'] == 'ns')
gi_filt_ns = np.logical_and(gi_filt, shear_table_rs['shear_type'] == 'ns')

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_rs_ns['wmom_band_mag_r'], shear_table_rs_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].scatter(shear_table_rs['wmom_band_mag_r'][ri_filt_ns], 
           shear_table_rs['wmom_color_mag_r-i'][ri_filt_ns], 
           marker='.', s=point_size) #red sequence galaxies
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_rs_ns['wmom_band_mag_g'], shear_table_rs_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) # all galaxies  
axes[1].scatter(shear_table_rs['wmom_band_mag_g'][gi_filt_ns], 
           shear_table_rs['wmom_color_mag_g-i'][gi_filt_ns], 
           marker='.', s=point_size)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([19,24])

plt.show()

Cut out the RS galaxies identified above.

In [ ]:
shear_table_wl = shear_table[shear_table['deg_sep'] < 0.5] # catalog for WL measurements
print("Number of rows in ns after applying < 0.5 deg from center: ", len(shear_table_wl))

In [ ]:
rs_hi_ri = r_up_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
ri_filt = shear_table_wl['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_wl['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_wl['wmom_band_mag_r'] < 23

rs_hi_gi = g_up_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
gi_filt = shear_table_wl['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_wl['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_wl['wmom_band_mag_g'] < 23

# ns only, for plotting
ri_filt_ns = np.logical_and(ri_filt, shear_table_wl['shear_type'] == 'ns')
gi_filt_ns = np.logical_and(gi_filt, shear_table_wl['shear_type'] == 'ns')

shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type'] == 'ns']

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) 
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].set_ylim([-0.5,3.5])
axes[1].set_xlim([19,24.5])

plt.show()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].scatter(shear_table_wl['wmom_band_mag_r'][ri_filt_ns], 
           shear_table_wl['wmom_color_mag_r-i'][ri_filt_ns], 
           marker='.', s=point_size) #red sequence galaxies
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) # all galaxies  
axes[1].scatter(shear_table_wl['wmom_band_mag_g'][gi_filt_ns], 
           shear_table_wl['wmom_color_mag_g-i'][gi_filt_ns], 
           marker='.', s=point_size)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([19,24])

plt.show()

In [ ]:
rs_filt = np.logical_and(ri_filt, gi_filt)
rs_filt_ns = np.logical_and(rs_filt, shear_table_wl['shear_type'] == 'ns')

RS_id_list = shear_table_wl['id'][rs_filt]
RS_id_list_ns = shear_table_wl_ns['id'][rs_filt_ns] # for plotting

In [ ]:
print(len(RS_id_list))
print(len(RS_id_list_ns)) # should be ~20% of above line

In [ ]:
# Filter out rows where the 'dataid' column matches any value in RS_id_list
shear_table_wl = shear_table_wl[~shear_table_wl['id'].isin(RS_id_list)]
shear_table_wl_ns = shear_table_wl_ns[~shear_table_wl_ns['id'].isin(RS_id_list_ns)]

In [ ]:
print(len(shear_table_wl))
print(len(shear_table_wl_ns))

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) 
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].set_ylim([-0.5,3.5])
axes[1].set_xlim([19,24.5])

plt.show()

In [ ]:
ra = shear_table_wl_ns['ra']
dec = shear_table_wl_ns['dec']
e1 = shear_table_wl_ns['wmom_g_1']
e2 = shear_table_wl_ns['wmom_g_2']
# e_err = TBD

In [ ]:
len(ra)

In [ ]:
plt.scatter(ra, dec, marker='.', s=0.2)
plt.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')

### Check lines of overdensity

Lines of overdensity: imperfect overlap of objects between patches/tracts.

(Outdated, but code might be useful later)

In [ ]:
# patch_list = []

# for ref in butler.registry.queryDatasets('deepCoaddCell', collections=cell_collection, band='i'):
#     patch_list.append(butler.query_datasets('deepCoaddCell', 
#                                                  collections=cell_collection,
#                                                  skymap = 'lsst_cells_v1',
#                                                  band = 'i',
#                                                  tract=ref.dataId['tract'],
#                                                  patch=ref.dataId['patch'])[0])

In [ ]:
# segs = []

# for ref in patch_list:
    
#     coadd = butler.get('deepCoaddCell',
#                       collections=cell_collection,
#                       skymap = 'lsst_cells_v1',
#                       band = 'i',
#                       tract=ref.dataId['tract'],
#                       patch=ref.dataId['patch'])

#     wcs = coadd.wcs
#     bbox = coadd.inner_bbox

#     coadd_corners = coadd.inner_bbox.getCorners()

#     for index, corner in enumerate(coadd_corners):
#         corner_coord_start = wcs.pixelToSky(corner.getX(), corner.getY())
#         if index < 3:
#             corner_coord_end = wcs.pixelToSky(coadd_corners[index+1].getX(), coadd_corners[index+1].getY())
#         else:
#             corner_coord_end = wcs.pixelToSky(coadd_corners[0].getX(), coadd_corners[0].getY())
    
#         start_ra = corner_coord_start[0].asDegrees()
#         start_dec = corner_coord_start[1].asDegrees()
    
#         end_ra = corner_coord_end[0].asDegrees()
#         end_dec = corner_coord_end[1].asDegrees()

#         segs.append(((start_ra, start_dec), (end_ra, end_dec)))

#     del coadd
#     gc.collect()

In [ ]:
# plt.scatter(ra, dec, marker='.', s=0.2)
# plt.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')
# for seg in segs:
#     plt.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.4)
# plt.show()

## Quality cuts

Using cuts from [Yamamoto 2024](https://arxiv.org/abs/2501.05665) for
- Star/galaxy separation
- S/N
- Size (visually check first if needed)
- Flux
- Color

These may need to be updated provided this is a different dataset, but should be a good starting point.

(It looks like the metadetect flags applied already take these into account?)

**Note: the Yamamoto paper cut for object size is definited differently than the Metadetect columns, so the object size cut from the DES code is used instead https://github.com/des-science/des-y6utils/blob/main/des_y6utils/mdet.py#L268**

In [ ]:
print("Object size ratio rows removed: ", len(shear_table)-len(shear_table[shear_table['wmom_T_ratio']>1.2]))
print("S/N ratio rows removed: ", len(shear_table)-len(shear_table[shear_table['wmom_s2n']>10]))
print("Size rows removed: ", len(shear_table)-len(shear_table[shear_table['wmom_T']<20]))
print("m_frac rows removed: ", len(shear_table)-len(shear_table[shear_table['mfrac']<0.1]))
print("faint in i-band removed: ", len(shear_table)-len(shear_table[shear_table['wmom_band_mag_i']<24.5]))
print("r-i flux rows removed: ", 
      len(shear_table)-len(shear_table[(shear_table['wmom_band_mag_r']-shear_table['wmom_band_mag_i']).abs()<5]))

In [ ]:
q_cuts = shear_table['wmom_T_ratio']>1.2
q_cuts &= shear_table['wmom_s2n']>10
q_cuts &= shear_table['wmom_T']<20
q_cuts &= shear_table['mfrac']<0.1
q_cuts &= shear_table['wmom_band_mag_i']<24.5
q_cuts &= (shear_table['wmom_band_mag_r']-shear_table['wmom_band_mag_i']).abs()<5

shear_table = shear_table[q_cuts]

print("Number of rows after applying quality cuts: ", len(shear_table))

# Looking at shear outputs

## Check shear types for each object

Each object is detected and measured separately for each sheared/unsheared image. The catalogs will not necessarily be the same but should be close in number of objects.

In [ ]:
# split catalog by shear type
shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type']=='ns']
shear_table_wl_1p = shear_table_wl[shear_table_wl['shear_type']=='1p']
shear_table_wl_1m = shear_table_wl[shear_table_wl['shear_type']=='1m']
shear_table_wl_2p = shear_table_wl[shear_table_wl['shear_type']=='2p']
shear_table_wl_2m = shear_table_wl[shear_table_wl['shear_type']=='2m']

In [ ]:
print("Number of shear type 'ns': ", len(shear_table_wl_ns))
print("Number of shear type '1p': ", len(shear_table_wl_1p))
print("Number of shear type '1m': ", len(shear_table_wl_1m))
print("Number of shear type '2p': ", len(shear_table_wl_2p))
print("Number of shear type '2m': ", len(shear_table_wl_2m))

## Check g1/g2 and PSF ellipticities

Check that the PSF and object ellipticities average to ~0

In [ ]:
# print mean values
print("Mean of psfrec_g_1: ", shear_table['psfrec_g_1'].median())
print("Mean of psfrec_g_2: ", shear_table['psfrec_g_2'].median())
print("Mean of wmom_g_1: ", shear_table['wmom_g_1'].median())
print("Mean of wmom_g_2: ", shear_table['wmom_g_2'].median())

In [ ]:
n_bins = 50

fig, axs = plt.subplots(1, 4, sharey=True, tight_layout=True)

axs[0].hist(shear_table['psfrec_g_1'], bins=n_bins)
axs[0].set_title('psfrec_g_1')
axs[1].hist(shear_table['psfrec_g_2'], bins=n_bins)
axs[1].set_title('psfrec_g_2')
axs[2].hist(shear_table['wmom_g_1'], bins=n_bins)
axs[2].set_title('wmom_g_1')
axs[3].hist(shear_table['wmom_g_2'], bins=n_bins)
axs[3].set_title('wmom_g_2')

fig.suptitle("Distribution of ellipticities for PSF and WMOM")

plt.show()

In [ ]:
n_bins = 50

fig, axs = plt.subplots(1, 4, sharey=True, tight_layout=True)

axs[0].hist(shear_table['psfrec_g_1'], bins=n_bins)
axs[0].set_yscale('log')
axs[0].set_title('psfrec_g_1')
axs[1].hist(shear_table['psfrec_g_2'], bins=n_bins)
axs[1].set_title('psfrec_g_2')
axs[1].set_yscale('log')
axs[2].hist(shear_table['wmom_g_1'], bins=n_bins)
axs[2].set_title('wmom_g_1')
axs[2].set_yscale('log')
axs[3].hist(shear_table['wmom_g_2'], bins=n_bins)
axs[3].set_title('wmom_g_2')
axs[3].set_yscale('log')

fig.suptitle("Distribution of ellipticities for PSF and WMOM")

plt.show()

PSG g1 appears less biased than last time, though is still pretty obiously biased. (Previously, 1 band with 4 whole tracts, had -0.25.). However, this does not seem to present a problem with the shape measurements.

## Determining tangential & cross shear

In [ ]:
# Radial binning, either in Mpc or degrees
# bins_mpc = clmm.make_bins(0.7,5,nbins=5, method='evenlog10width')
bins_deg = clmm.make_bins(0.08,0.5,nbins=6, method='evenlog10width')
bins_mpc = cosmo.eval_da(0.22) * bins_deg * np.pi/180

# define distance bins
dig_rad_bins_ns = np.digitize(shear_table_wl_ns['deg_sep'], bins_deg)
dig_mpc_bins_ns = np.digitize(shear_table_wl_ns['mpc_sep'], bins_mpc)

bin_mid_mpc = [(0.5 * (bins_mpc[i] + bins_mpc[i+1])) for i in range(len(bins_mpc)-1)]
bin_mid_deg = [(0.5 * (bins_deg[i] + bins_deg[i+1])) for i in range(len(bins_deg)-1)]

shear_diff = 0.02

Definitions of tangential and cross shear used:
$$ \gamma_t = -\gamma_1\cos(2\phi)-\gamma_2\sin(2\phi) $$
$$ \gamma_\times = \gamma_1\sin(2\phi)-\gamma_2\cos(2\phi) $$

### Individual Phi

This section will attempt to measure $\gamma_t$ and $\gamma_\times$ by:
- Calculate individual $\phi$ for each galaxy
- Calculate $\gamma_t$ and $\gamma_\times$ for each galaxy
- Calculate the response R from all galaxies, no binning
- Apply R to averaged / binned $\gamma_t,\gamma_\times$

In [ ]:
p1_mean = shear_table_wl_1p['wmom_g_1'].mean()
m1_mean = shear_table_wl_1m['wmom_g_1'].mean()
p2_mean = shear_table_wl_2p['wmom_g_2'].mean()
m2_mean = shear_table_wl_2m['wmom_g_2'].mean()

r_matrix = [[0, 0],[0, 0]]

r_matrix[0][0] = (p1_mean - m1_mean) / shear_diff
# r_matrix[0][1] = (p2_mean - m2_mean) / shear_diff # ignore?
# r_matrix[1][0] = (p1_mean - m1_mean) / shear_diff
r_matrix[1][1] = (p2_mean - m2_mean) / shear_diff

r_matrix_inv = inv(r_matrix)

In [ ]:
# bin tangential and cross shears by radial bins
tan_cross_shears = np.zeros((len(bins_mpc)-1, 2)) # binned tangential and cross shear
tan_errs = []
cross_errs = []

for i in range(0, len(bins_mpc)-1):
    bin_filt_ns = dig_mpc_bins_ns == i+1

    # print number of galaxies in each bin
    print("Rows in bin ", i, " :", len(shear_table_wl_ns['shear_t'][bin_filt_ns]))

    # calulcate mean t and x shears
    mean_t_shear = shear_table_wl_ns['shear_t'][bin_filt_ns].mean() # mean g1
    mean_x_shear = shear_table_wl_ns['shear_x'][bin_filt_ns].mean() # mean g2

    # calculate errors
    tan_err = stats.bootstrap([shear_table_wl_ns['shear_t'][bin_filt_ns]], np.mean).confidence_interval
    cross_err = stats.bootstrap([shear_table_wl_ns['shear_x'][bin_filt_ns]], np.mean).confidence_interval

    # apply calibration 
    shear_cal = r_matrix_inv.dot([mean_t_shear, mean_x_shear])
    # shear_cal = [mean_t_shear, mean_x_shear]

    err_upper = r_matrix_inv.dot([tan_err.high, cross_err.high])
    err_lower = r_matrix_inv.dot([tan_err.low, cross_err.low])

    tan_err = [err_lower[0], err_upper[0]]
    cross_err = [err_lower[1], err_upper[1]]

    tan_cross_shears[i] = shear_cal
    tan_errs.append(tan_err)
    cross_errs.append(cross_err)

tan_errs = np.array(tan_errs)
cross_errs = np.array(cross_errs)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(6,6))

point_size = 40
point_alpha = 1

# g1 calibrated
axes.scatter(bin_mid_mpc, tan_cross_shears[:,0],
             marker='.', s=point_size, label='tangential shear')
axes.plot(bin_mid_mpc, tan_cross_shears[:,0], '-o')
axes.vlines(bin_mid_mpc, tan_errs[:,0], tan_errs[:,1], colors='blue')

cross_axis = np.add(bin_mid_mpc, 0.05) # add offset to visually differentiate

axes.scatter(cross_axis, tan_cross_shears[:,1],
             marker='.', s=point_size, label='cross shear')
axes.plot(cross_axis, tan_cross_shears[:,1], '-o')
axes.vlines(cross_axis, cross_errs[:,0], cross_errs[:,1], colors='orange')

axes.set_ylabel("reduced shear")
axes.set_xlabel("distance from BCG in Mpc")

axes.legend(loc="upper right")
plt.axhline(0.0, color='k', ls=':')

plt.show()

Currently error bars are from a boostrapping method with 0.95 confidence intervals, with the errors also calibrated with R. 

A better method is likely to take the STDV g / sqrt number of galaxies, and then propogate that error through R.

#### Getting NFW Model with CLMM Package

In [ ]:
galcat = GCData()
galcat['ra'] = shear_table_wl_ns['ra']
galcat['dec'] = shear_table_wl_ns['dec']
galcat['e1'] = shear_table_wl_ns['wmom_g_1'] * 2
galcat['e2'] = shear_table_wl_ns['wmom_g_2'] * 2

galcat['z'] = np.zeros(len(shear_table_wl_ns)) # CLMM needs a redshift column for the source, even if not used

In [ ]:
cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, coordinate_system='euclidean')

In [ ]:
gc_object1.compute_galaxy_weights(
        shape_component1="e1",
        shape_component2="e2",
        use_shape_error=False,
        use_shape_noise=True,
        weight_name="w_ls",
        cosmo=cosmo,
        add=True,
    )

In [ ]:
moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
moo.set_mass(1.0e15)

z_cl = gc_object1.z

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(6,6))

point_size = 40
point_alpha = 1

# g1 calibrated
axes.scatter(bin_mid_mpc, tan_cross_shears[:,0],
             marker='.', s=point_size, label='tangential shear')
axes.plot(bin_mid_mpc, tan_cross_shears[:,0], '-o')
axes.vlines(bin_mid_mpc, tan_errs[:,0], tan_errs[:,1], colors='blue')

cross_axis = np.add(bin_mid_mpc, 0.05) # add offset to visually differentiate

axes.scatter(cross_axis, tan_cross_shears[:,1],
             marker='.', s=point_size, label='cross shear')
axes.plot(cross_axis, tan_cross_shears[:,1], '-o')
axes.vlines(cross_axis, cross_errs[:,0], cross_errs[:,1], colors='orange')

plt.axhline(0.0, color='k', ls=':')

plt.plot(rproj, gt_z, label='NFW (model, not fit), M200m=1e15 Msun, c=4, n(z)=SRD', ls=':')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
plt.ylim([-0.03,0.08])
plt.xlim([0.7,7])
#plt.yscale('log')
axes.set_xlabel('R [Mpc]')
axes.set_ylabel("reduced shear")
axes.legend(loc=1)

plt.show()

Shears look good except for the closest bin. More work to do in error bars, null tests, and incorporating photo-z information.